# Lab | Handling Data Imbalance in Classification Models

For this lab and in the next lessons we will use the dataset 'Healthcare For All' building a model to predict who will donate (TargetB) and how much they will give (TargetD) (will be used for lab on Friday). You will be using files_for_lab/learningSet.csv file which you have already downloaded from class.

Scenario

You are revisiting the Healthcare for All Case Study. You are provided with this historical data about Donors and how much they donated. Your task is to build a machine learning model that will help the company identify people who are more likely to donate and then try to predict the donation amount.

Instructions

In this lab, we will first take a look at the degree of imbalance in the data and correct it using the techniques we learned in the class.

Here is the list of steps to be followed (building a simple model without balancing the data):

Import the required libraries and modules that you would need.

Read that data into Python and call the dataframe donors.

Check the datatypes of all the columns in the data.

Check for null values in the dataframe. Replace the null values using the methods learned in class.

Split the data into numerical and catagorical. Decide if any columns need their dtype changed.

Concatenate numerical and categorical back together again for your X dataframe. Designate the Target as y.

Split the data into a training set and a test set.
Split further into train_num and train_cat. Also test_num and test_cat.
Scale the features either by using normalizer or a standard scaler. (train_num, test_num)
Encode the categorical features using One-Hot Encoding or Ordinal Encoding. (train_cat, test_cat)
fit only on train data transform both train and test
again re-concatenate train_num and train_cat as X_train as well as test_num and test_cat as X_test
Fit a logistic regression model on the training data.
Check the accuracy on the test data.
Note: So far we have not balanced the data.

Managing imbalance in the dataset

Check for the imbalance.
Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
Each time fit the model and see how the accuracy of the model has changed.

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [7]:
categorical = pd.read_csv('/Users/igorhufnagel/Desktop/Ironhack/LABS/Week 16/Day 3/lab-handling-data-imbalance-classification/categorical_cleaned.csv')
numerical = pd.read_csv('/Users/igorhufnagel/Desktop/Ironhack/LABS/Week 16/Day 3/lab-handling-data-imbalance-classification/numerical_cleaned.csv')
targets = pd.read_csv('/Users/igorhufnagel/Desktop/Ironhack/LABS/Week 16/Day 3/lab-handling-data-imbalance-classification/Y.csv')

In [9]:
targets=targets.drop(columns="TARGET_D")

In [12]:
scaler = MinMaxScaler()

for col in numerical.columns:
    numerical[col] = scaler.fit_transform(numerical[[col]])

numerical.head()

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,...,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0.426523,0.000000,0.382286,0.608247,0.000000,1.000000,0.000000,0.000000,0.393939,0.343434,...,0.863139,0.010,0.045226,0.927939,0.003676,0.006465,0.498045,0.0,1.000000,0.622951
1,0.784946,0.000014,0.535736,0.463918,0.857143,1.000000,0.066390,0.000000,0.151515,0.555556,...,0.913321,0.025,0.045226,0.969489,0.016544,0.014399,0.774510,0.0,0.333333,0.000000
2,0.498208,0.000014,0.000000,0.624862,0.428571,0.111111,0.008299,0.000000,0.202020,0.292929,...,0.774179,0.005,0.045226,0.937311,0.011029,0.006204,0.078617,1.0,1.000000,0.967213
3,0.283154,0.000000,0.288465,0.711340,0.142857,0.444444,0.008299,0.000000,0.232323,0.141414,...,0.867245,0.010,0.045226,0.906175,0.008272,0.005534,0.899764,1.0,1.000000,0.655738
4,0.211470,0.000000,0.206076,0.793814,0.428571,0.222222,0.248963,0.010101,0.282828,0.090909,...,0.953923,0.015,0.492462,0.822972,0.012868,0.005586,0.037079,1.0,0.333333,0.409836


In [11]:
categorical.nunique()

STATE          12
ZIP         19938
CLUSTER        53
HOMEOWNR        2
DATASRCE        3
RFA_2          14
RFA_2R          1
RFA_2A          4
GEOCODE2        4
DOMAIN_A        5
DOMAIN_B        4
dtype: int64

In [13]:
categorical=categorical.drop(columns="ZIP")
categorical.nunique()

STATE       12
CLUSTER     53
HOMEOWNR     2
DATASRCE     3
RFA_2       14
RFA_2R       1
RFA_2A       4
GEOCODE2     4
DOMAIN_A     5
DOMAIN_B     4
dtype: int64

In [15]:
one_hot_names = []
for col in categorical.columns:
    col_uniques = sorted(categorical[col].astype(str).unique())
    for unique in col_uniques:
        one_hot_names.append(col+"_"+unique)
        
categorical = pd.DataFrame(OneHotEncoder().fit_transform(categorical.astype(str)).toarray())
categorical.columns = one_hot_names
categorical.head()

,STATE_CA_0.0,STATE_CA_1.0,STATE_FL_0.0,STATE_FL_1.0,STATE_GA_0.0,STATE_GA_1.0,STATE_IL_0.0,STATE_IL_1.0,STATE_IN_0.0,STATE_IN_1.0,...,DOMAIN_A_U_0.0,DOMAIN_A_U_1.0,DOMAIN_B_1_0.0,DOMAIN_B_1_1.0,DOMAIN_B_2_0.0,DOMAIN_B_2_1.0,DOMAIN_B_3_0.0,DOMAIN_B_3_1.0,DOMAIN_B_4_0.0,DOMAIN_B_4_1.0
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


In [16]:
X = pd.concat([numerical, categorical], axis=1)
y = targets

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
X_train.head()

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,...,DOMAIN_A_U_0.0,DOMAIN_A_U_1.0,DOMAIN_B_1_0.0,DOMAIN_B_1_1.0,DOMAIN_B_2_0.0,DOMAIN_B_2_1.0,DOMAIN_B_3_0.0,DOMAIN_B_3_1.0,DOMAIN_B_4_0.0,DOMAIN_B_4_1.0
85225,0.928315,0.000389,0.000000,0.624862,0.000000,1.000000,0.000000,0.0,0.292929,0.363636,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
70004,0.498208,0.000000,0.350257,0.649485,0.571429,1.000000,0.000000,0.0,0.232323,0.232323,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
88133,0.354839,0.000014,0.360556,0.639175,1.000000,0.666667,0.008299,0.0,0.252525,0.272727,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
79106,0.354839,0.000028,0.267868,0.731959,0.285714,1.000000,0.000000,0.0,0.343434,0.191919,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
35476,0.426523,0.000000,0.000000,0.624862,0.142857,1.000000,0.000000,0.0,0.373737,0.464646,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


In [19]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [20]:
accuracy = model.score(X_test, y_test)
accuracy

0.9487501965099827

In [21]:
targets.value_counts()

TARGET_B
0           90569
1            4843
dtype: int64

In [24]:
# OVERSAMPLING

smote = SMOTE()

x_resampled,y_resampled=smote.fit_resample(X,targets)
y_resampled.value_counts()

TARGET_B
0           90569
1           90569
dtype: int64

In [25]:
X_train, X_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.6045876117919841

In [27]:
# UNDERSAMPLING

RUS=RandomUnderSampler(random_state=0)
x_resampled,y_resampled=RUS.fit_resample(X,targets)

y_resampled.value_counts()

TARGET_B
0           4843
1           4843
dtype: int64

In [28]:
X_train, X_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.5784313725490197

In [29]:
# TOMEKLINKS

from imblearn.under_sampling import TomekLinks

tl = TomekLinks(sampling_strategy='majority')
X_tl, y_tl=tl.fit_resample(X,y)
y_tl.value_counts()

TARGET_B
0           88529
1            4843
dtype: int64

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_tl, y_tl, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9493975903614458